In [1]:
import tensorflow as tf
import numpy as np

In [2]:
X = []
t = 1
steps = 1
for i in range(t):
    with open("matrices/view" + str(i) + ".npy", 'rb') as f:
        X.append(tf.cast(tf.sparse.from_dense(np.load(f)), dtype=tf.float32))

In [6]:
v_shape = X[0].shape
alpha = 1
beta = 1000
gamma = 0.01
wv = [1.0 / t for i in range(t)]
I = tf.eye(v_shape[0], dtype=tf.float32)
Z = [0 for i in range(t)]
S = tf.eye(v_shape[0], dtype=tf.float32)

In [7]:
for step in range(steps):
    S = tf.where(S > 0, S, tf.zeros(v_shape))
    S = (S + tf.transpose(S)) / 2
    L = tf.linalg.diag(tf.math.reduce_sum(S, 0)) - S
    for xi in range(t):
        Xv = tf.sparse.to_dense(X[xi])
        Z[xi] = tf.sparse.from_dense(
            tf.linalg.matmul(
                tf.linalg.inv(tf.linalg.matmul(Xv, Xv, transpose_a=True, a_is_sparse=True, b_is_sparse=True) +
                              I * alpha +
                              I * beta * wv[xi]),
                S * beta * wv[xi] + tf.linalg.matmul(Xv, Xv, transpose_a=True, a_is_sparse=True, b_is_sparse=True)))

In [8]:
print(Z[0])
# # x = X[0]
# x = tf.sparse.to_dense(X[0])  -1
# print(x)
# y = tf.where(x > 0, x, tf.zeros(v_shape))
# print(y)

SparseTensor(indices=tf.Tensor(
[[   0    0]
 [   0    1]
 [   0    2]
 ...
 [4754 4754]
 [4755 4755]
 [4756 4756]], shape=(51517, 2), dtype=int64), values=tf.Tensor(
[9.9902481e-01 5.5085193e-06 5.5047940e-06 ... 9.9900097e-01 9.9900097e-01
 9.9900097e-01], shape=(51517,), dtype=float32), dense_shape=tf.Tensor([4757 4757], shape=(2,), dtype=int64))
